In [26]:
import pandas as pd
import dask.dataframe as dd
import praw
from dask.diagnostics import ProgressBar
from tqdm import tqdm
import hashlib
from PIL import Image
import requests
from io import BytesIO
from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.spark.set_environ import set_azure_env
import os
import torch
from datetime import datetime
import pandas as pd
from pmaw import PushshiftAPI
import time
set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from shared_code.utility.scripts.blip_caption import BlipCaption

pbar = ProgressBar()
pbar.register()

tqdm.pandas()
tqdm.pandas(desc="global")

from tqdm.dask import TqdmCallback
cb = TqdmCallback(desc="global")
cb.register()

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [27]:
%%time
 #["SFWRedheads","sfwpetite","SFWNextDoorGirls", "celebrities", "wallstreetbets", "fatsquirrelhate","CityPorn","EarthPorn","memes","spaceporn","trippinthroughtime","fatsquirrelhate","oldladiesbakingpies","itookapicture", "celebrities", "realasians", "KoreanHotties", "prettyasiangirls","AsianOfficeLady", "mildlypenis","AsianOfficeLady"]
subs = ["realasians"]

CPU times: total: 0 ns
Wall time: 0 ns
CPU times: total: 0 ns
Wall time: 0 ns


In [28]:
%%time

extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
display(extant_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,128z5ns,celebrities,A-Sexy-Name,Rachel Weisz,cars are parked on the side of the road in the...,f09b1c0c5dea50c934081a04c83d4d33,/r/celebrities/comments/128z5ns/rachel_weisz/,https://i.redd.it/nv80afddacra1.jpg,f09b1c0c5dea50c934081a04c83d4d33.jpg,D:\data\images\celebrities\f09b1c0c5dea50c9340...,D:\data\images\celebrities\thumbnail\f09b1c0c5...,True,False
1,11yk8db,prettyasiangirls,MisoShiru520,Library,blonde woman with blonde hair and tattoos on h...,9d5b7331b941ecf3fdbc8ad322303825,/r/prettyasiangirls/comments/11yk8db/library/,https://i.redd.it/ht4d2wuusapa1.jpg,9d5b7331b941ecf3fdbc8ad322303825.jpg,D:\data\images\prettyasiangirls\9d5b7331b941ec...,D:\data\images\prettyasiangirls\thumbnail\9d5b...,True,False
2,11hfioi,KoreanHotties,Majestic_Painter8660,Underboob bikini,a man with a beard and a beard sitting in fron...,a154a56b1b87cf3a0bc93600c86e4585,/r/KoreanHotties/comments/11hfioi/underboob_bi...,https://i.redd.it/cimr17q84jla1.jpg,a154a56b1b87cf3a0bc93600c86e4585.jpg,D:\data\images\KoreanHotties\a154a56b1b87cf3a0...,D:\data\images\KoreanHotties\thumbnail\a154a56...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29597,11oz7fi,mildlypenis,Extra-Lie489,Lenis,there is a small piece of metal with two small...,7604c3e55ce8dbf238520683d6e71f89,/r/mildlypenis/comments/11oz7fi/lenis/,https://i.redd.it/pskj3q0ao8na1.jpg,7604c3e55ce8dbf238520683d6e71f89.jpg,D:\data\images\mildlypenis\7604c3e55ce8dbf2385...,D:\data\images\mildlypenis\thumbnail\7604c3e55...,True,False
29598,11oz73d,mildlypenis,DeimosLV,My friend is making a stuffed animal and this ...,a close up of a blue knitted bagueth on a chair,389f8aa0fb191bdeba08c8d2cfff8ce2,/r/mildlypenis/comments/11oz73d/my_friend_is_m...,https://i.redd.it/gemqhq67o8na1.jpg,389f8aa0fb191bdeba08c8d2cfff8ce2.jpg,D:\data\images\mildlypenis\389f8aa0fb191bdeba0...,D:\data\images\mildlypenis\thumbnail\389f8aa0f...,True,False
29599,11oyzm3,mildlypenis,Mymemesareswell,This weird Mickey keychain hand,someone is holding a key chain with a mickey m...,87563ac0c8ddeeb0ce47bc915b99308b,/r/mildlypenis/comments/11oyzm3/this_weird_mic...,https://i.redd.it/vrivt8rom8na1.jpg,87563ac0c8ddeeb0ce47bc915b99308b.jpg,D:\data\images\mildlypenis\87563ac0c8ddeeb0ce4...,D:\data\images\mildlypenis\thumbnail\87563ac0c...,True,False
29600,11oyvx1,mildlypenis,LucindaBobinda,Parking lot pp,a close up of a black and white photo of a bla...,88b596be6de959bd29eed4e8946bb476,/r/mildlypenis/comments/11oyvx1/parking_lot_pp/,https://i.redd.it/w1vebizwl8na1.jpg,88b596be6de959bd29eed4e8946bb476.jpg,D:\data\images\mildlypenis\88b596be6de959bd29e...,D:\data\images\mildlypenis\thumbnail\88b596be6...,True,False


CPU times: total: 641 ms
Wall time: 1.73 s


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,128z5ns,celebrities,A-Sexy-Name,Rachel Weisz,cars are parked on the side of the road in the...,f09b1c0c5dea50c934081a04c83d4d33,/r/celebrities/comments/128z5ns/rachel_weisz/,https://i.redd.it/nv80afddacra1.jpg,f09b1c0c5dea50c934081a04c83d4d33.jpg,D:\data\images\celebrities\f09b1c0c5dea50c9340...,D:\data\images\celebrities\thumbnail\f09b1c0c5...,True,False
1,11yk8db,prettyasiangirls,MisoShiru520,Library,blonde woman with blonde hair and tattoos on h...,9d5b7331b941ecf3fdbc8ad322303825,/r/prettyasiangirls/comments/11yk8db/library/,https://i.redd.it/ht4d2wuusapa1.jpg,9d5b7331b941ecf3fdbc8ad322303825.jpg,D:\data\images\prettyasiangirls\9d5b7331b941ec...,D:\data\images\prettyasiangirls\thumbnail\9d5b...,True,False
2,11hfioi,KoreanHotties,Majestic_Painter8660,Underboob bikini,a man with a beard and a beard sitting in fron...,a154a56b1b87cf3a0bc93600c86e4585,/r/KoreanHotties/comments/11hfioi/underboob_bi...,https://i.redd.it/cimr17q84jla1.jpg,a154a56b1b87cf3a0bc93600c86e4585.jpg,D:\data\images\KoreanHotties\a154a56b1b87cf3a0...,D:\data\images\KoreanHotties\thumbnail\a154a56...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29713,12fu3tr,realasians,Impossible-Act6792,Cute,there is a woman laying on a bed with a white ...,10b92a383163fc494c78382a13942705,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,10b92a383163fc494c78382a13942705.jpg,D:\data\images\realasians\10b92a383163fc494c78...,D:\data\images\realasians\thumbnail\10b92a3831...,True,False
29714,12ftdwo,realasians,sfsfw,Trying on new shirt,araffed woman sitting on the floor taking a se...,35a3e44afa15890b9f62f4e5d8895e50,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,35a3e44afa15890b9f62f4e5d8895e50.jpg,D:\data\images\realasians\35a3e44afa15890b9f62...,D:\data\images\realasians\thumbnail\35a3e44afa...,True,False
29715,12ft1ri,realasians,SKTminion,Cruise gal,araffe woman in a bikini on a cruise ship,ace0f33ad4affe4e0f112bf8dd1f36fe,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,ace0f33ad4affe4e0f112bf8dd1f36fe.jpg,D:\data\images\realasians\ace0f33ad4affe4e0f11...,D:\data\images\realasians\thumbnail\ace0f33ad4...,True,False
29716,12fst4o,realasians,dahyunscat,Choose one,two women in bikinis standing on a beach at night,959ea8c1956b7ff8a0a4eb61bb161436,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,959ea8c1956b7ff8a0a4eb61bb161436.jpg,D:\data\images\realasians\959ea8c1956b7ff8a0a4...,D:\data\images\realasians\thumbnail\959ea8c195...,True,False


CPU times: total: 375 ms
Wall time: 743 ms


In [29]:
reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

api: PushshiftAPI = PushshiftAPI(praw=reddit, num_workers=12)

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [30]:
def filter_by_image_post(item) -> bool:
	return not (item['selftext'].__contains__('[removed]') or item['selftext'].__contains__('[deleted]') and not item[
		'url'].endswith('.jpg'))

In [31]:
def fetch_image(x: object) -> object:
	try:
		time.sleep(0.1)
		url = x['original_url']
		subreddit = x['subreddit']
		response = requests.get(url)
		md5 = hashlib.md5(response.content).hexdigest()
		os.makedirs(f"D:\\data\\images\\{subreddit}", exist_ok=True)
		out_path = f"D:\\data\\images\\{subreddit}\\{md5}.jpg"

		if os.path.exists(out_path):
			return out_path

		if md5 in list(extant_data['hash']):
			return out_path

		if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
			raw_image = Image.open(BytesIO(response.content))
			raw_image.save(out_path)
			raw_image.close()
			return out_path
		else:
			return ""
	except Exception as e:
		print(e)
		return ""

In [32]:
def make_thumbnail(x: object) -> str:
	image_path = x['path']
	md5 = x['hash']
	subreddit = x['subreddit']
	os.makedirs(f"D:\\data\\images\\{subreddit}\\thumbnail", exist_ok=True)
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5}.jpg"
	if not os.path.exists(image_path):
		return ""
	if os.path.exists(out_path):
		return out_path
	img = Image.open(image_path)
	try:
		copied_image = img.copy()
		result_copy = copied_image.resize((512, 512))
		result_copy.save(out_path)
		result_copy.close()
		img.close()
		return out_path
	except Exception as e:
		print(e)
		return ""
	finally:
		img.close()

In [33]:
def exists(x: object) -> bool:
	try:
		image_path = os.path.exists(x['path'])
		thumbnail_path = os.path.exists(x['thumbnail_path'])
		return image_path and thumbnail_path
	except Exception as e:
		print(e)
		return False

In [34]:
def split_hash(x: object) -> str:
	try:
		name = x['image_name']
		return name.split('.')[0]
	except Exception as e:
		print(e)
		return ""

In [35]:
def caption_image(x: object, blip_captioning: BlipCaption) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if len(resulting_caption) > 5:
			return resulting_caption

		resulting_caption = blip_captioning.caption_image(path)
		return resulting_caption

	except Exception as e:
		print(e)
		return ""
	finally:
		pass

In [36]:
def get_image_name(x: object) -> str:
	try:
		path = x['path']
		if path == "":
			return ""
		return os.path.basename(path)
	except Exception as e:
		print(e)
		return ""

In [37]:
#@title Calculate Time Intervals For Training

start_date = "2023-04-08"
end_date = datetime.today().timestamp()
day_in_seconds = 86400

start_date_time_stamp: int = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())

end_date_time_stamp: int = int(end_date)

total_number_days_since_start = (end_date_time_stamp - start_date_time_stamp) // day_in_seconds

temp = start_date_time_stamp

intervals = []
while temp < end_date_time_stamp:
    temp = temp + day_in_seconds
    intervals.append(temp)

print(f"Total Number Of Intervals: {len(intervals)}\n")

Total Number Of Intervals: 2

Total Number Of Intervals: 2



In [13]:
from tqdm import notebook
posts = []
for date in notebook.tqdm(intervals, desc=f"Downloading Reddit Data {intervals[0]} - {intervals[-1]}"):
    post_in_interval = api.search_submissions(subreddit=subs[0], limit=100, until=end_date_time_stamp, after=start_date_time_stamp, cache_dir='D:\\cache\\', mem_safe=True, filter_fn=filter_by_image_post)
    posts.extend(post_in_interval)

foo=pd.DataFrame(data=posts)

display(foo)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,num_crossposts,media,is_video,_fetched,_comments_by_id,post_hint,preview,is_gallery,media_metadata,gallery_data
0,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_u4kgo17v,False,None,0,...,0,None,False,False,{},NaN,NaN,NaN,NaN,NaN
1,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_820i830e,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
2,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_998p7yvn,False,None,0,...,0,None,False,False,{},NaN,NaN,True,"{'lsvomih88xsa1': {'status': 'valid', 'e': 'Im...","{'items': [{'media_id': '27fafih88xsa1', 'id':..."
3,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_789j3g7j,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
4,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_vlf07z8o,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748BF100>,None,realasians,,t2_izy0b,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
164,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748BF100>,None,realasians,,t2_5br9wc0x,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://extern...,NaN,NaN,NaN
165,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748BF100>,None,realasians,,t2_2p195ro0,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
166,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748BF100>,None,realasians,,t2_82rkecd7,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN


In [14]:
%%time

df = pd.DataFrame([item for item in posts])
display(df)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,num_crossposts,media,is_video,_fetched,_comments_by_id,post_hint,preview,is_gallery,media_metadata,gallery_data
0,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_u4kgo17v,False,None,0,...,0,None,False,False,{},NaN,NaN,NaN,NaN,NaN
1,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_820i830e,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
2,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_998p7yvn,False,None,0,...,0,None,False,False,{},NaN,NaN,True,"{'lsvomih88xsa1': {'status': 'valid', 'e': 'Im...","{'items': [{'media_id': '27fafih88xsa1', 'id':..."
3,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_789j3g7j,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
4,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748CB1F0>,None,realasians,,t2_vlf07z8o,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748BF100>,None,realasians,,t2_izy0b,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
164,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748BF100>,None,realasians,,t2_5br9wc0x,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://extern...,NaN,NaN,NaN
165,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748BF100>,None,realasians,,t2_2p195ro0,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN
166,2048,confidence,<praw.reddit.Reddit object at 0x000001AE748BF100>,None,realasians,,t2_82rkecd7,False,None,0,...,0,None,False,False,{},image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN,NaN


CPU times: total: 109 ms
Wall time: 313 ms


In [15]:
%%time

initial = pd.DataFrame({}, columns=schema.names)
initial['id'] = df['id']
initial['subreddit'] = df['subreddit']
initial['author'] = df['author']
initial['title'] = df['title']
initial['caption'] = ""
initial['hash'] = ""
initial['permalink'] = df['permalink']
initial['original_url'] = df['url']
initial['image_name'] = ""
initial['path'] = ""
initial['thumbnail_path'] = ""
initial['exists'] = False
initial['curated'] = False

filtered_initial = initial.where(initial['original_url'].str.endswith('.jpg')).dropna(how='all').reset_index().drop('index', axis=1)

display(filtered_initial)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,12gvw57,realasians,phoshodoe,Like the view?,,,/r/realasians/comments/12gvw57/like_the_view/,https://i.redd.it/2j7jg8b0uysa1.jpg,,,,False,False
1,12gurwb,realasians,notaverage22,Tank tops are always in fashion,,,/r/realasians/comments/12gurwb/tank_tops_are_a...,https://i.redd.it/yrlht9nxmysa1.jpg,,,,False,False
2,12gul8c,realasians,ricecooker276,KS2,,,/r/realasians/comments/12gul8c/ks2/,https://i.redd.it/1p32g7l94xsa1.jpg,,,,False,False
3,12gtcel,realasians,OOO00O0O000,🌍,,,/r/realasians/comments/12gtcel/_/,https://i.redd.it/i276gl0rdysa1.jpg,,,,False,False
4,12gt7fa,realasians,smolnmighty,Lazy Sundays,,,/r/realasians/comments/12gt7fa/lazy_sundays/,https://i.redd.it/5ljho7qtcysa1.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,12fu3tr,realasians,Impossible-Act6792,Cute,,,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,,,,False,False
112,12ftdwo,realasians,sfsfw,Trying on new shirt,,,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,,,,False,False
113,12ft1ri,realasians,SKTminion,Cruise gal,,,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,,,,False,False
114,12fst4o,realasians,dahyunscat,Choose one,,,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,,,,False,False


CPU times: total: 62.5 ms
Wall time: 250 ms


In [16]:
%%time

new_entries = filtered_initial.where(~filtered_initial['id'].isin(extant_data['id'])).dropna(how='all').reset_index().drop('index', axis=1)

display(new_entries)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,12gvw57,realasians,phoshodoe,Like the view?,,,/r/realasians/comments/12gvw57/like_the_view/,https://i.redd.it/2j7jg8b0uysa1.jpg,,,,False,False
1,12gurwb,realasians,notaverage22,Tank tops are always in fashion,,,/r/realasians/comments/12gurwb/tank_tops_are_a...,https://i.redd.it/yrlht9nxmysa1.jpg,,,,False,False
2,12gul8c,realasians,ricecooker276,KS2,,,/r/realasians/comments/12gul8c/ks2/,https://i.redd.it/1p32g7l94xsa1.jpg,,,,False,False
3,12gtcel,realasians,OOO00O0O000,🌍,,,/r/realasians/comments/12gtcel/_/,https://i.redd.it/i276gl0rdysa1.jpg,,,,False,False
4,12gt7fa,realasians,smolnmighty,Lazy Sundays,,,/r/realasians/comments/12gt7fa/lazy_sundays/,https://i.redd.it/5ljho7qtcysa1.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,12fu3tr,realasians,Impossible-Act6792,Cute,,,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,,,,False,False
112,12ftdwo,realasians,sfsfw,Trying on new shirt,,,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,,,,False,False
113,12ft1ri,realasians,SKTminion,Cruise gal,,,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,,,,False,False
114,12fst4o,realasians,dahyunscat,Choose one,,,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,,,,False,False


CPU times: total: 62.5 ms
Wall time: 141 ms


In [17]:
%%time

new_entries_1 = new_entries.copy()

tqdm.pandas(desc="Fetch-Image")
with ProgressBar():
	new_entries_1['path'] = new_entries_1.progress_apply(lambda x: fetch_image(x), axis=1)

tqdm.pandas(desc="Get-Image-Name")
with ProgressBar():
	new_entries_1['image_name'] = new_entries_1.progress_apply(lambda x: get_image_name(x), axis=1)

tqdm.pandas(desc="Split-Hash")
with ProgressBar():
	new_entries_1['hash'] = new_entries_1.progress_apply(lambda x: split_hash(x), axis=1)

display(new_entries_1)

Split-Hash: 100%|██████████| 116/116 [00:00<?, ?it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,12gvw57,realasians,phoshodoe,Like the view?,,29d35ab3ec2380fc8dedc9fd802c5ce2,/r/realasians/comments/12gvw57/like_the_view/,https://i.redd.it/2j7jg8b0uysa1.jpg,29d35ab3ec2380fc8dedc9fd802c5ce2.jpg,D:\data\images\realasians\29d35ab3ec2380fc8ded...,,False,False
1,12gurwb,realasians,notaverage22,Tank tops are always in fashion,,4971bc097716eb86431b748eb77fdf81,/r/realasians/comments/12gurwb/tank_tops_are_a...,https://i.redd.it/yrlht9nxmysa1.jpg,4971bc097716eb86431b748eb77fdf81.jpg,D:\data\images\realasians\4971bc097716eb86431b...,,False,False
2,12gul8c,realasians,ricecooker276,KS2,,1a72d23ec23a94213d1d5d88118da7d8,/r/realasians/comments/12gul8c/ks2/,https://i.redd.it/1p32g7l94xsa1.jpg,1a72d23ec23a94213d1d5d88118da7d8.jpg,D:\data\images\realasians\1a72d23ec23a94213d1d...,,False,False
3,12gtcel,realasians,OOO00O0O000,🌍,,44ac6b32fcd9c60aaf8aba61d4d1a9a9,/r/realasians/comments/12gtcel/_/,https://i.redd.it/i276gl0rdysa1.jpg,44ac6b32fcd9c60aaf8aba61d4d1a9a9.jpg,D:\data\images\realasians\44ac6b32fcd9c60aaf8a...,,False,False
4,12gt7fa,realasians,smolnmighty,Lazy Sundays,,51a58273487e67a9cdc5c7d2be473e12,/r/realasians/comments/12gt7fa/lazy_sundays/,https://i.redd.it/5ljho7qtcysa1.jpg,51a58273487e67a9cdc5c7d2be473e12.jpg,D:\data\images\realasians\51a58273487e67a9cdc5...,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,12fu3tr,realasians,Impossible-Act6792,Cute,,10b92a383163fc494c78382a13942705,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,10b92a383163fc494c78382a13942705.jpg,D:\data\images\realasians\10b92a383163fc494c78...,,False,False
112,12ftdwo,realasians,sfsfw,Trying on new shirt,,35a3e44afa15890b9f62f4e5d8895e50,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,35a3e44afa15890b9f62f4e5d8895e50.jpg,D:\data\images\realasians\35a3e44afa15890b9f62...,,False,False
113,12ft1ri,realasians,SKTminion,Cruise gal,,ace0f33ad4affe4e0f112bf8dd1f36fe,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,ace0f33ad4affe4e0f112bf8dd1f36fe.jpg,D:\data\images\realasians\ace0f33ad4affe4e0f11...,,False,False
114,12fst4o,realasians,dahyunscat,Choose one,,959ea8c1956b7ff8a0a4eb61bb161436,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,959ea8c1956b7ff8a0a4eb61bb161436.jpg,D:\data\images\realasians\959ea8c1956b7ff8a0a4...,,False,False


CPU times: total: 11.1 s
Wall time: 42.9 s


In [18]:
%%time

new_entries_2 = new_entries_1.copy()

tqdm.pandas(desc="Create-Thumbnail")
with ProgressBar():
	new_entries_2['thumbnail_path'] = new_entries_2.progress_apply(lambda x: make_thumbnail(x), axis=1)

tqdm.pandas(desc="Checking-Exists")
with ProgressBar():
	new_entries_2['exists'] = new_entries_2.progress_apply(lambda x: exists(x), axis=1)

display(new_entries_2)

Checking-Exists: 100%|██████████| 116/116 [00:00<00:00, 1484.83it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,12gvw57,realasians,phoshodoe,Like the view?,,29d35ab3ec2380fc8dedc9fd802c5ce2,/r/realasians/comments/12gvw57/like_the_view/,https://i.redd.it/2j7jg8b0uysa1.jpg,29d35ab3ec2380fc8dedc9fd802c5ce2.jpg,D:\data\images\realasians\29d35ab3ec2380fc8ded...,D:\data\images\realasians\thumbnail\29d35ab3ec...,True,False
1,12gurwb,realasians,notaverage22,Tank tops are always in fashion,,4971bc097716eb86431b748eb77fdf81,/r/realasians/comments/12gurwb/tank_tops_are_a...,https://i.redd.it/yrlht9nxmysa1.jpg,4971bc097716eb86431b748eb77fdf81.jpg,D:\data\images\realasians\4971bc097716eb86431b...,D:\data\images\realasians\thumbnail\4971bc0977...,True,False
2,12gul8c,realasians,ricecooker276,KS2,,1a72d23ec23a94213d1d5d88118da7d8,/r/realasians/comments/12gul8c/ks2/,https://i.redd.it/1p32g7l94xsa1.jpg,1a72d23ec23a94213d1d5d88118da7d8.jpg,D:\data\images\realasians\1a72d23ec23a94213d1d...,D:\data\images\realasians\thumbnail\1a72d23ec2...,True,False
3,12gtcel,realasians,OOO00O0O000,🌍,,44ac6b32fcd9c60aaf8aba61d4d1a9a9,/r/realasians/comments/12gtcel/_/,https://i.redd.it/i276gl0rdysa1.jpg,44ac6b32fcd9c60aaf8aba61d4d1a9a9.jpg,D:\data\images\realasians\44ac6b32fcd9c60aaf8a...,D:\data\images\realasians\thumbnail\44ac6b32fc...,True,False
4,12gt7fa,realasians,smolnmighty,Lazy Sundays,,51a58273487e67a9cdc5c7d2be473e12,/r/realasians/comments/12gt7fa/lazy_sundays/,https://i.redd.it/5ljho7qtcysa1.jpg,51a58273487e67a9cdc5c7d2be473e12.jpg,D:\data\images\realasians\51a58273487e67a9cdc5...,D:\data\images\realasians\thumbnail\51a5827348...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,12fu3tr,realasians,Impossible-Act6792,Cute,,10b92a383163fc494c78382a13942705,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,10b92a383163fc494c78382a13942705.jpg,D:\data\images\realasians\10b92a383163fc494c78...,D:\data\images\realasians\thumbnail\10b92a3831...,True,False
112,12ftdwo,realasians,sfsfw,Trying on new shirt,,35a3e44afa15890b9f62f4e5d8895e50,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,35a3e44afa15890b9f62f4e5d8895e50.jpg,D:\data\images\realasians\35a3e44afa15890b9f62...,D:\data\images\realasians\thumbnail\35a3e44afa...,True,False
113,12ft1ri,realasians,SKTminion,Cruise gal,,ace0f33ad4affe4e0f112bf8dd1f36fe,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,ace0f33ad4affe4e0f112bf8dd1f36fe.jpg,D:\data\images\realasians\ace0f33ad4affe4e0f11...,D:\data\images\realasians\thumbnail\ace0f33ad4...,True,False
114,12fst4o,realasians,dahyunscat,Choose one,,959ea8c1956b7ff8a0a4eb61bb161436,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,959ea8c1956b7ff8a0a4eb61bb161436.jpg,D:\data\images\realasians\959ea8c1956b7ff8a0a4...,D:\data\images\realasians\thumbnail\959ea8c195...,True,False


CPU times: total: 6.61 s
Wall time: 9.39 s


In [19]:
%%time

blip = BlipCaption(1)

CPU times: total: 1min
Wall time: 1min 17s


In [20]:
%%time

new_entries_final = new_entries_2.copy()
torch.cuda.empty_cache()

with pbar:
	ddf = dd.from_pandas(new_entries_final, npartitions=6)
	new_entries_final['caption'] = ddf.apply(lambda x: caption_image(x, blip), meta=('str', object), axis=1).compute()

torch.cuda.empty_cache()

display(new_entries_final)

[                                        ] | 0% Completed | 1.57 ms

global:   0%|          | 0/6 [00:00<?, ?it/s]

[                                        ] | 0% Completed | 58.83 sms

D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[####################                    ] | 50% Completed | 310.95 s

...

[#################################       ] | 83% Completed | 344.77 s

D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[########################################] | 100% Completed | 354.32 s


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,12gvw57,realasians,phoshodoe,Like the view?,araffe woman in a white bikini on a boat in th...,29d35ab3ec2380fc8dedc9fd802c5ce2,/r/realasians/comments/12gvw57/like_the_view/,https://i.redd.it/2j7jg8b0uysa1.jpg,29d35ab3ec2380fc8dedc9fd802c5ce2.jpg,D:\data\images\realasians\29d35ab3ec2380fc8ded...,D:\data\images\realasians\thumbnail\29d35ab3ec...,True,False
1,12gurwb,realasians,notaverage22,Tank tops are always in fashion,a close up of a woman in a black top holding a...,4971bc097716eb86431b748eb77fdf81,/r/realasians/comments/12gurwb/tank_tops_are_a...,https://i.redd.it/yrlht9nxmysa1.jpg,4971bc097716eb86431b748eb77fdf81.jpg,D:\data\images\realasians\4971bc097716eb86431b...,D:\data\images\realasians\thumbnail\4971bc0977...,True,False
2,12gul8c,realasians,ricecooker276,KS2,there is a woman taking a selfie in a mirror,1a72d23ec23a94213d1d5d88118da7d8,/r/realasians/comments/12gul8c/ks2/,https://i.redd.it/1p32g7l94xsa1.jpg,1a72d23ec23a94213d1d5d88118da7d8.jpg,D:\data\images\realasians\1a72d23ec23a94213d1d...,D:\data\images\realasians\thumbnail\1a72d23ec2...,True,False
3,12gtcel,realasians,OOO00O0O000,🌍,araffe woman in a bikini and a hat taking a se...,44ac6b32fcd9c60aaf8aba61d4d1a9a9,/r/realasians/comments/12gtcel/_/,https://i.redd.it/i276gl0rdysa1.jpg,44ac6b32fcd9c60aaf8aba61d4d1a9a9.jpg,D:\data\images\realasians\44ac6b32fcd9c60aaf8a...,D:\data\images\realasians\thumbnail\44ac6b32fc...,True,False
4,12gt7fa,realasians,smolnmighty,Lazy Sundays,a close up of a person in a black panties with...,51a58273487e67a9cdc5c7d2be473e12,/r/realasians/comments/12gt7fa/lazy_sundays/,https://i.redd.it/5ljho7qtcysa1.jpg,51a58273487e67a9cdc5c7d2be473e12.jpg,D:\data\images\realasians\51a58273487e67a9cdc5...,D:\data\images\realasians\thumbnail\51a5827348...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,12fu3tr,realasians,Impossible-Act6792,Cute,there is a woman laying on a bed with a white ...,10b92a383163fc494c78382a13942705,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,10b92a383163fc494c78382a13942705.jpg,D:\data\images\realasians\10b92a383163fc494c78...,D:\data\images\realasians\thumbnail\10b92a3831...,True,False
112,12ftdwo,realasians,sfsfw,Trying on new shirt,araffed woman sitting on the floor taking a se...,35a3e44afa15890b9f62f4e5d8895e50,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,35a3e44afa15890b9f62f4e5d8895e50.jpg,D:\data\images\realasians\35a3e44afa15890b9f62...,D:\data\images\realasians\thumbnail\35a3e44afa...,True,False
113,12ft1ri,realasians,SKTminion,Cruise gal,araffe woman in a bikini on a cruise ship,ace0f33ad4affe4e0f112bf8dd1f36fe,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,ace0f33ad4affe4e0f112bf8dd1f36fe.jpg,D:\data\images\realasians\ace0f33ad4affe4e0f11...,D:\data\images\realasians\thumbnail\ace0f33ad4...,True,False
114,12fst4o,realasians,dahyunscat,Choose one,two women in bikinis standing on a beach at night,959ea8c1956b7ff8a0a4eb61bb161436,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,959ea8c1956b7ff8a0a4eb61bb161436.jpg,D:\data\images\realasians\959ea8c1956b7ff8a0a4...,D:\data\images\realasians\thumbnail\959ea8c195...,True,False


CPU times: total: 6min 11s
Wall time: 6min 48s


In [21]:
%%time

tqdm.pandas(desc="Filtering")
writeable_entries = new_entries_final\
	.where(~new_entries_final['id'].isin(extant_data['id']))\
	.dropna(how='all')\
	.reset_index()\
	.drop('index', axis=1)

display(writeable_entries)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,12gvw57,realasians,phoshodoe,Like the view?,araffe woman in a white bikini on a boat in th...,29d35ab3ec2380fc8dedc9fd802c5ce2,/r/realasians/comments/12gvw57/like_the_view/,https://i.redd.it/2j7jg8b0uysa1.jpg,29d35ab3ec2380fc8dedc9fd802c5ce2.jpg,D:\data\images\realasians\29d35ab3ec2380fc8ded...,D:\data\images\realasians\thumbnail\29d35ab3ec...,True,False
1,12gurwb,realasians,notaverage22,Tank tops are always in fashion,a close up of a woman in a black top holding a...,4971bc097716eb86431b748eb77fdf81,/r/realasians/comments/12gurwb/tank_tops_are_a...,https://i.redd.it/yrlht9nxmysa1.jpg,4971bc097716eb86431b748eb77fdf81.jpg,D:\data\images\realasians\4971bc097716eb86431b...,D:\data\images\realasians\thumbnail\4971bc0977...,True,False
2,12gul8c,realasians,ricecooker276,KS2,there is a woman taking a selfie in a mirror,1a72d23ec23a94213d1d5d88118da7d8,/r/realasians/comments/12gul8c/ks2/,https://i.redd.it/1p32g7l94xsa1.jpg,1a72d23ec23a94213d1d5d88118da7d8.jpg,D:\data\images\realasians\1a72d23ec23a94213d1d...,D:\data\images\realasians\thumbnail\1a72d23ec2...,True,False
3,12gtcel,realasians,OOO00O0O000,🌍,araffe woman in a bikini and a hat taking a se...,44ac6b32fcd9c60aaf8aba61d4d1a9a9,/r/realasians/comments/12gtcel/_/,https://i.redd.it/i276gl0rdysa1.jpg,44ac6b32fcd9c60aaf8aba61d4d1a9a9.jpg,D:\data\images\realasians\44ac6b32fcd9c60aaf8a...,D:\data\images\realasians\thumbnail\44ac6b32fc...,True,False
4,12gt7fa,realasians,smolnmighty,Lazy Sundays,a close up of a person in a black panties with...,51a58273487e67a9cdc5c7d2be473e12,/r/realasians/comments/12gt7fa/lazy_sundays/,https://i.redd.it/5ljho7qtcysa1.jpg,51a58273487e67a9cdc5c7d2be473e12.jpg,D:\data\images\realasians\51a58273487e67a9cdc5...,D:\data\images\realasians\thumbnail\51a5827348...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,12fu3tr,realasians,Impossible-Act6792,Cute,there is a woman laying on a bed with a white ...,10b92a383163fc494c78382a13942705,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,10b92a383163fc494c78382a13942705.jpg,D:\data\images\realasians\10b92a383163fc494c78...,D:\data\images\realasians\thumbnail\10b92a3831...,True,False
112,12ftdwo,realasians,sfsfw,Trying on new shirt,araffed woman sitting on the floor taking a se...,35a3e44afa15890b9f62f4e5d8895e50,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,35a3e44afa15890b9f62f4e5d8895e50.jpg,D:\data\images\realasians\35a3e44afa15890b9f62...,D:\data\images\realasians\thumbnail\35a3e44afa...,True,False
113,12ft1ri,realasians,SKTminion,Cruise gal,araffe woman in a bikini on a cruise ship,ace0f33ad4affe4e0f112bf8dd1f36fe,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,ace0f33ad4affe4e0f112bf8dd1f36fe.jpg,D:\data\images\realasians\ace0f33ad4affe4e0f11...,D:\data\images\realasians\thumbnail\ace0f33ad4...,True,False
114,12fst4o,realasians,dahyunscat,Choose one,two women in bikinis standing on a beach at night,959ea8c1956b7ff8a0a4eb61bb161436,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,959ea8c1956b7ff8a0a4eb61bb161436.jpg,D:\data\images\realasians\959ea8c1956b7ff8a0a4...,D:\data\images\realasians\thumbnail\959ea8c195...,True,False


CPU times: total: 46.9 ms
Wall time: 243 ms


In [22]:
%%time

combined_result = pd.concat([extant_data, writeable_entries])

display(combined_result)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,128z5ns,celebrities,A-Sexy-Name,Rachel Weisz,cars are parked on the side of the road in the...,f09b1c0c5dea50c934081a04c83d4d33,/r/celebrities/comments/128z5ns/rachel_weisz/,https://i.redd.it/nv80afddacra1.jpg,f09b1c0c5dea50c934081a04c83d4d33.jpg,D:\data\images\celebrities\f09b1c0c5dea50c9340...,D:\data\images\celebrities\thumbnail\f09b1c0c5...,True,False
1,11yk8db,prettyasiangirls,MisoShiru520,Library,blonde woman with blonde hair and tattoos on h...,9d5b7331b941ecf3fdbc8ad322303825,/r/prettyasiangirls/comments/11yk8db/library/,https://i.redd.it/ht4d2wuusapa1.jpg,9d5b7331b941ecf3fdbc8ad322303825.jpg,D:\data\images\prettyasiangirls\9d5b7331b941ec...,D:\data\images\prettyasiangirls\thumbnail\9d5b...,True,False
2,11hfioi,KoreanHotties,Majestic_Painter8660,Underboob bikini,a man with a beard and a beard sitting in fron...,a154a56b1b87cf3a0bc93600c86e4585,/r/KoreanHotties/comments/11hfioi/underboob_bi...,https://i.redd.it/cimr17q84jla1.jpg,a154a56b1b87cf3a0bc93600c86e4585.jpg,D:\data\images\KoreanHotties\a154a56b1b87cf3a0...,D:\data\images\KoreanHotties\thumbnail\a154a56...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,12fu3tr,realasians,Impossible-Act6792,Cute,there is a woman laying on a bed with a white ...,10b92a383163fc494c78382a13942705,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,10b92a383163fc494c78382a13942705.jpg,D:\data\images\realasians\10b92a383163fc494c78...,D:\data\images\realasians\thumbnail\10b92a3831...,True,False
112,12ftdwo,realasians,sfsfw,Trying on new shirt,araffed woman sitting on the floor taking a se...,35a3e44afa15890b9f62f4e5d8895e50,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,35a3e44afa15890b9f62f4e5d8895e50.jpg,D:\data\images\realasians\35a3e44afa15890b9f62...,D:\data\images\realasians\thumbnail\35a3e44afa...,True,False
113,12ft1ri,realasians,SKTminion,Cruise gal,araffe woman in a bikini on a cruise ship,ace0f33ad4affe4e0f112bf8dd1f36fe,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,ace0f33ad4affe4e0f112bf8dd1f36fe.jpg,D:\data\images\realasians\ace0f33ad4affe4e0f11...,D:\data\images\realasians\thumbnail\ace0f33ad4...,True,False
114,12fst4o,realasians,dahyunscat,Choose one,two women in bikinis standing on a beach at night,959ea8c1956b7ff8a0a4eb61bb161436,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,959ea8c1956b7ff8a0a4eb61bb161436.jpg,D:\data\images\realasians\959ea8c1956b7ff8a0a4...,D:\data\images\realasians\thumbnail\959ea8c195...,True,False


CPU times: total: 62.5 ms
Wall time: 187 ms


In [23]:
combined_result.reindex()
combined_result.to_csv('processed_raw_data.csv', index=False)
f = AzureFileStorageAdapter('data').get_file_storage()
f.put('processed_raw_data.csv','data/processed_raw_data.csv', overwrite=True)

[None]

In [24]:
final = pd.read_csv(f.open('data/processed_raw_data.csv'))
final.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

In [25]:
final = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(final)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,128z5ns,celebrities,A-Sexy-Name,Rachel Weisz,cars are parked on the side of the road in the...,f09b1c0c5dea50c934081a04c83d4d33,/r/celebrities/comments/128z5ns/rachel_weisz/,https://i.redd.it/nv80afddacra1.jpg,f09b1c0c5dea50c934081a04c83d4d33.jpg,D:\data\images\celebrities\f09b1c0c5dea50c9340...,D:\data\images\celebrities\thumbnail\f09b1c0c5...,True,False
1,11yk8db,prettyasiangirls,MisoShiru520,Library,blonde woman with blonde hair and tattoos on h...,9d5b7331b941ecf3fdbc8ad322303825,/r/prettyasiangirls/comments/11yk8db/library/,https://i.redd.it/ht4d2wuusapa1.jpg,9d5b7331b941ecf3fdbc8ad322303825.jpg,D:\data\images\prettyasiangirls\9d5b7331b941ec...,D:\data\images\prettyasiangirls\thumbnail\9d5b...,True,False
2,11hfioi,KoreanHotties,Majestic_Painter8660,Underboob bikini,a man with a beard and a beard sitting in fron...,a154a56b1b87cf3a0bc93600c86e4585,/r/KoreanHotties/comments/11hfioi/underboob_bi...,https://i.redd.it/cimr17q84jla1.jpg,a154a56b1b87cf3a0bc93600c86e4585.jpg,D:\data\images\KoreanHotties\a154a56b1b87cf3a0...,D:\data\images\KoreanHotties\thumbnail\a154a56...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29713,12fu3tr,realasians,Impossible-Act6792,Cute,there is a woman laying on a bed with a white ...,10b92a383163fc494c78382a13942705,/r/realasians/comments/12fu3tr/cute/,https://i.redd.it/61rm6nl4vqsa1.jpg,10b92a383163fc494c78382a13942705.jpg,D:\data\images\realasians\10b92a383163fc494c78...,D:\data\images\realasians\thumbnail\10b92a3831...,True,False
29714,12ftdwo,realasians,sfsfw,Trying on new shirt,araffed woman sitting on the floor taking a se...,35a3e44afa15890b9f62f4e5d8895e50,/r/realasians/comments/12ftdwo/trying_on_new_s...,https://i.redd.it/xtnkp0q29psa1.jpg,35a3e44afa15890b9f62f4e5d8895e50.jpg,D:\data\images\realasians\35a3e44afa15890b9f62...,D:\data\images\realasians\thumbnail\35a3e44afa...,True,False
29715,12ft1ri,realasians,SKTminion,Cruise gal,araffe woman in a bikini on a cruise ship,ace0f33ad4affe4e0f112bf8dd1f36fe,/r/realasians/comments/12ft1ri/cruise_gal/,https://i.imgur.com/TPq6yuj.jpg,ace0f33ad4affe4e0f112bf8dd1f36fe.jpg,D:\data\images\realasians\ace0f33ad4affe4e0f11...,D:\data\images\realasians\thumbnail\ace0f33ad4...,True,False
29716,12fst4o,realasians,dahyunscat,Choose one,two women in bikinis standing on a beach at night,959ea8c1956b7ff8a0a4eb61bb161436,/r/realasians/comments/12fst4o/choose_one/,https://i.redd.it/ye63r1c1nqsa1.jpg,959ea8c1956b7ff8a0a4eb61bb161436.jpg,D:\data\images\realasians\959ea8c1956b7ff8a0a4...,D:\data\images\realasians\thumbnail\959ea8c195...,True,False
